# ANN DEPLOYMENT WITH GRADIO - {"CUSTOMER CHURN" DATASET}

## 1. Importing Modules

In [1]:
import numpy as np
import pandas as pd

import tensorflow
from tensorflow import keras

from pickle import dump, load

import gradio as gr

import warnings
warnings.filterwarnings('ignore')

from sklearn import set_config
set_config(display='diagram')

## 2. Loading Data and Model

### 2.1 Loading Xtrain, FE Pipe, and Model

In [2]:
X = load(open('Xtr_prod.pkl','rb'))

pipe = load(open('cc_pipe2_te.pkl','rb'))

pred_mdl = tensorflow.keras.models.load_model('mdl_prod_fe_ht.keras')
pred_mdl.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                2240      
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2,273
Trainable params: 2,273
Non-trainable params: 0
_________________________________________________________________


### 2.2 Loading Test Data

In [3]:
te = pd.read_pickle('cc_test_pp.pkl')

print(f'Shape of the test dataset : {te.shape}')

Shape of the test dataset : (100, 20)


### 2.3 Preparing App Samples from Test Data

In [4]:
te_df = te.sample(10)
te_df

,state,account_length,area_code,international_plan,voice_mail_plan,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls,churn
3712,MI,130,area_code_415,no,no,0,143.8,76,24.45,274.8,103,23.36,194.7,105,8.76,12.3,1,3.32,0,no
3550,MI,110,area_code_415,no,no,0,94.1,127,16.00,283.0,97,24.06,168.7,69,7.59,10.3,6,2.78,2,no
3229,KS,124,area_code_510,no,no,0,247.2,84,42.02,161.9,130,13.76,170.3,73,7.66,6.1,6,1.65,1,no
292,AR,179,area_code_415,yes,yes,38,220.1,78,37.42,234.3,71,19.92,237.3,85,10.68,10.1,4,2.73,4,no
3508,AL,76,area_code_415,no,no,0,222.0,99,37.74,215.9,122,18.35,204.1,131,9.18,8.2,5,2.21,3,yes
365,MI,81,area_code_415,no,no,0,153.5,99,26.10,197.6,102,16.80,198.5,86,8.93,6.3,2,1.70,2,no
2030,MI,108,area_code_408,no,yes,32,209.5,108,35.62,109.6,64,9.32,189.7,145,8.54,9.1,6,2.46,6,yes
412,ME,75,area_code_408,yes,no,0,211.3,61,35.92,105.6,119,8.98,175.9,63,7.92,9.7,4,2.62,4,yes
974,AK,173,area_code_510,no,no,0,172.5,78,29.33,142.6,91,12.12,102.0,63,4.59,10.9,2,2.94,0,no
995,WA,29,area_code_415,no,no,0,157.4,122,26.76,145.0,75,12.33,281.8,92,12.68,9.3,2,2.51,1,no


In [5]:
print('App Samples without Labels ...')
Xte = te_df.drop(columns='churn')
gr_samples = Xte.values.tolist()

App Samples without Labels ...


## 3. Gradio App Development

### 3.1 Gradio App Prediction Function

In [6]:
def cc_prediction(state, acc_len, area_code, intl_pln, voi_pln, vmail_msg, day_mins, day_calls, day_chrg,
                 eve_mins, eve_calls, eve_chrg, ngt_mins, ngt_calls, ngt_chrg, intl_mins, intl_calls, intl_chrg, cust_serv_calls):
    
    state = str(state)
    acc_len = int(acc_len)
    area_code = str(area_code)
    intl_pln = str(intl_pln)
    voi_pln = str(voi_pln)
    vmail_msg = int(vmail_msg)
    day_mins = float(day_mins)
    day_calls = int(day_calls)
    day_chrg = float(day_chrg)
    eve_mins = float(eve_mins)    
    eve_calls = int(eve_calls)
    eve_chrg = float(eve_chrg)
    ngt_mins = float(ngt_mins)
    ngt_calls = int(ngt_calls)    
    ngt_chrg = float(ngt_chrg)
    intl_mins = float(intl_mins)
    intl_calls = int(intl_calls)
    intl_chrg = float(intl_chrg) 
    cust_serv_calls = int(cust_serv_calls) 
    
    data = np.array([state,acc_len,area_code,intl_pln,voi_pln,vmail_msg,day_mins,day_calls,day_chrg,eve_mins,eve_calls,eve_chrg,ngt_mins,ngt_calls,ngt_chrg,intl_mins,intl_calls,intl_chrg,cust_serv_calls],dtype=object).reshape(1,19)
    #print(data)
    data_fe = pipe.transform(data)
    #print(data_fe)
    data_fin = data_fe.astype('float')
    #print(data_fin)
    res = pred_mdl.predict(data_fin)
    #print(res)
    
    res = np.where(res[0][0] >= 0.5, 1, 0)
    #print(res)
    if res == 1:
        return f'Prediction : Customer will Churn --> {res}'
    else:
        return f'Prediction : Customer will NOT Churn --> {res}'

In [7]:
res = cc_prediction('MO', 74, 'area_code_415', 'no', 'no', 0, 172.1, 105, 29.26, 211.7, 99, 17.99, 182.2, 105, 8.2, 11.6, 6, 3.13, 1)
print(res)

1/1 [==============================] - 0s 150ms/step
Prediction : Customer will NOT Churn --> 0


### 3.2 Gradio App Interface for Input and Output

In [16]:
iface = gr.Interface(cc_prediction, 
        inputs=[
                gr.Dropdown(X[X.columns[0]].unique().tolist(), label='State'),
                gr.Slider(int(X[X.columns[1]].min()),int(X[X.columns[1]].max()),20, step=1, label='Account Length'), 
                gr.Dropdown(X[X.columns[2]].unique().tolist(), label='Area Code'),
                gr.Dropdown(X[X.columns[3]].unique().tolist(), label='International Plan'),
                gr.Dropdown(X[X.columns[4]].unique().tolist(), label='Voice Mail Plan'),
                gr.Slider(int(X[X.columns[5]].min()),int(X[X.columns[5]].max()),20, step=1, label='Voice Mail Messages'),
                gr.Textbox(default=None, label='Total Day Minutes'),
                gr.Slider(int(X[X.columns[7]].min()),int(X[X.columns[7]].max()),20, step=1, label='Total Day Calls'),
                gr.Textbox(default=None, label='Total Day Charges'),
                gr.Textbox(default=None, label='Total Evening Minutes'),
                gr.Slider(int(X[X.columns[10]].min()),int(X[X.columns[10]].max()),20, step=1, label='Total Evening Calls'),
                gr.Textbox(default=None, label='Total Evening Charges'),
                gr.Textbox(default=None, label='Total Night Minutes'),
                gr.Slider(int(X[X.columns[13]].min()),int(X[X.columns[13]].max()),20, step=1, label='Total Night Calls'),
                gr.Textbox(default=None, label='Total Night Charges'),
                gr.Textbox(default=None, label='Total International Minutes'),
                gr.Slider(int(X[X.columns[16]].min()),int(X[X.columns[16]].max()),20, step=1, label='Total International Calls'),
                gr.Textbox(default=None, label='Total International Charges'),
                gr.Textbox(default=None, label='Customer Service Calls')             
               ], 
        outputs="text",
        layout='horizontal', theme='soft', 
        title='BigMart Sales Predictor App', 
        description='Provide Details of the Item and Outlet', 
        article='Created by: Manish Sharma',
        examples=gr_samples) 

### 3.3 Gradio App Launch

In [17]:
te_df

,state,account_length,area_code,international_plan,voice_mail_plan,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls,churn
3712,MI,130,area_code_415,no,no,0,143.8,76,24.45,274.8,103,23.36,194.7,105,8.76,12.3,1,3.32,0,no
3550,MI,110,area_code_415,no,no,0,94.1,127,16.00,283.0,97,24.06,168.7,69,7.59,10.3,6,2.78,2,no
3229,KS,124,area_code_510,no,no,0,247.2,84,42.02,161.9,130,13.76,170.3,73,7.66,6.1,6,1.65,1,no
292,AR,179,area_code_415,yes,yes,38,220.1,78,37.42,234.3,71,19.92,237.3,85,10.68,10.1,4,2.73,4,no
3508,AL,76,area_code_415,no,no,0,222.0,99,37.74,215.9,122,18.35,204.1,131,9.18,8.2,5,2.21,3,yes
365,MI,81,area_code_415,no,no,0,153.5,99,26.10,197.6,102,16.80,198.5,86,8.93,6.3,2,1.70,2,no
2030,MI,108,area_code_408,no,yes,32,209.5,108,35.62,109.6,64,9.32,189.7,145,8.54,9.1,6,2.46,6,yes
412,ME,75,area_code_408,yes,no,0,211.3,61,35.92,105.6,119,8.98,175.9,63,7.92,9.7,4,2.62,4,yes
974,AK,173,area_code_510,no,no,0,172.5,78,29.33,142.6,91,12.12,102.0,63,4.59,10.9,2,2.94,0,no
995,WA,29,area_code_415,no,no,0,157.4,122,26.76,145.0,75,12.33,281.8,92,12.68,9.3,2,2.51,1,no


In [18]:
iface.launch(share=False, debug=True, width=1125, height=750)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.
